In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
# 1. Preprocess the data: Convert categorical columns to numbers
# We use one-hot encoding to convert 'sex', 'smoker', and 'region' into numerical format.
# This is crucial for the model to understand these features.
dataset = pd.get_dummies(dataset, columns=['sex', 'smoker', 'region'], prefix='', prefix_sep='')

# 2. Split the data into training (80%) and testing (20%) sets
# A random state is used for reproducibility.
train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)

# 3. Separate features from labels
# The 'expenses' column is what we want to predict (the label).
# All other columns are the features the model will use for prediction.
train_labels = train_dataset.pop('expenses')
test_labels = test_dataset.pop('expenses')

# 4. Normalize the feature data
# Normalization scales all numeric features to a similar range, which helps
# the neural network learn more effectively and quickly.
normalizer = layers.Normalization(axis=-1)
normalizer.adapt(np.array(train_dataset))

# 5. Build the neural network model
# We'll create a Sequential model with two hidden layers using the 'relu' activation
# function, and a final output layer with a single neuron for our regression output.
model = keras.Sequential([
      normalizer,
      layers.Dense(64, activation='relu'),
      layers.Dense(64, activation='relu'),
      layers.Dense(1)
  ])

# 6. Compile the model
# We configure the model for training.
# - The 'Adam' optimizer is an efficient choice for most problems.
# - The loss function is 'mean_absolute_error', which is what we want to minimize.
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
              loss='mean_absolute_error',
              metrics=['mae', 'mse'])

# 7. Train the model
# We fit the model to the training data for 150 epochs.
# A validation split of 20% helps us monitor for overfitting.
history = model.fit(
    train_dataset,
    train_labels,
    epochs=150,
    verbose=0, # Suppress training output
    validation_split=0.2)

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
